In [55]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [56]:
ROOT = "/content/drive/MyDrive/Colab Notebooks/DM/논문참조"

### 0. Installing Konlpy

In [3]:
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

Hit:1 http://archive.ubuntu.com/ubuntu focal InRelease
Get:2 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:4 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:5 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Get:10 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:11 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:12 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [3,175 kB]
Get:13 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1,344 kB]
Get:14 http://secu

In [4]:
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

env: JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"


In [5]:
pip install python-mecab-ko

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 573.9/573.9 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 44.6 MB/s eta 0:00:00


In [6]:
!curl -s https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/mecab-colab.sh | bash

--2023-05-19 17:21:48--  https://www.dropbox.com/s/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.66.18, 2620:100:6027:18::a27d:4812
Connecting to www.dropbox.com (www.dropbox.com)|162.125.66.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz [following]
--2023-05-19 17:21:48--  https://www.dropbox.com/s/dl/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc0abcb59a98465c30b8b7a8aad0.dl.dropboxusercontent.com/cd/0/get/B8XnKrz0oUc_B9ZA6e6YFK_nw215m2U9pMP9ZBk938dnMnKTMynAy2zK0p8X_NGMhb-owXZ5lu2mccHfBx9aS_EpoJ8YjyZJLZU6p_IT6tfe6RqD7eA6ExZRUm5-ndu6XDn3tyDJq9XZfh53vBLQ7f8W1s0sf4ZWvBG1LYC9YuKN28moeWE0ix4_ws_bUkgE7Ro/file?dl=1# [following]
--2023-05-19 17:21:49--  https://uc0abcb59a98465c30b8b7a8aad0.dl.dropboxusercontent.com/cd/0/get/B8XnKrz0o

#### Test Code

In [7]:
import konlpy
from konlpy.tag import Kkma, Komoran, Hannanum, Okt
from konlpy.utils import pprint
from mecab import MeCab

In [8]:
mecab = MeCab()

In [9]:
sentence = "데이터마이닝 term project 코드입니다."
temp_X = mecab.morphs(sentence)
temp_X

['데이터', '마이닝', 'term', 'project', '코드', '입니다', '.']

### 1. Data

In [57]:
from tqdm import tqdm
import re
import pickle
import csv
import pandas as pd
import numpy as np

In [58]:
school = pd.read_csv(f'{ROOT}/data/교내 주제선택 분반_위도경도 통합.csv', encoding = 'cp949')
school.head()

,학교명,위치,학생수,예산(만원),매칭교과,강좌명,활동목표,위도,경도
0,경북체육중학교,경산시,26.0,700.0,과학,내가 닮고 싶은 과학자 이야기,1. 과학자가 들려주는 과학 이야기를 읽고 내가 닮고 싶은 과학자에 대한 자료를 ...,35.812226,128.738053
1,경북체육중학교,경산시,26.0,700.0,국어,문학에 빠지다,"1. 문학에 대한 이해를 높이고, 책 읽기를 통하여 문학에 대한 이해를 기를 수 ...",35.812226,128.738053
2,경북체육중학교,경산시,26.0,700.0,사회,영화 속 사회,"1. 일본군 위안부와 관련된 자료를 찾아 그 실상을 알고, 홍보 굿즈를 제작해 보...",35.812226,128.738053
3,경산여자중학교,경산시,281.0,NaN,국어,시 감상과 모방시 창작,1. 문학은 심미적 체험을 바탕으로 한 다양한 소통 활동임을 알고 문학 활동을 할...,35.817764,128.729945
4,경산여자중학교,경산시,281.0,NaN,사회,내가 만들어가는 민주정치,1. 민주 정치를 실현하기 위한 노력을 국가와 시민의 차원에서 탐구하여 설명할 수 ...,35.817764,128.729945


In [59]:
school['활동목표'].fillna("", inplace=True)

In [60]:
school['제목_활동'] = school['강좌명'] + school['활동목표']
school.head()

,학교명,위치,학생수,예산(만원),매칭교과,강좌명,활동목표,위도,경도,제목_활동
0,경북체육중학교,경산시,26.0,700.0,과학,내가 닮고 싶은 과학자 이야기,1. 과학자가 들려주는 과학 이야기를 읽고 내가 닮고 싶은 과학자에 대한 자료를 ...,35.812226,128.738053,내가 닮고 싶은 과학자 이야기 1. 과학자가 들려주는 과학 이야기를 읽고 내가 닮고...
1,경북체육중학교,경산시,26.0,700.0,국어,문학에 빠지다,"1. 문학에 대한 이해를 높이고, 책 읽기를 통하여 문학에 대한 이해를 기를 수 ...",35.812226,128.738053,"문학에 빠지다 1. 문학에 대한 이해를 높이고, 책 읽기를 통하여 문학에 대한 이해..."
2,경북체육중학교,경산시,26.0,700.0,사회,영화 속 사회,"1. 일본군 위안부와 관련된 자료를 찾아 그 실상을 알고, 홍보 굿즈를 제작해 보...",35.812226,128.738053,"영화 속 사회 1. 일본군 위안부와 관련된 자료를 찾아 그 실상을 알고, 홍보 굿즈..."
3,경산여자중학교,경산시,281.0,NaN,국어,시 감상과 모방시 창작,1. 문학은 심미적 체험을 바탕으로 한 다양한 소통 활동임을 알고 문학 활동을 할...,35.817764,128.729945,시 감상과 모방시 창작 1. 문학은 심미적 체험을 바탕으로 한 다양한 소통 활동임을...
4,경산여자중학교,경산시,281.0,NaN,사회,내가 만들어가는 민주정치,1. 민주 정치를 실현하기 위한 노력을 국가와 시민의 차원에서 탐구하여 설명할 수 ...,35.817764,128.729945,내가 만들어가는 민주정치1. 민주 정치를 실현하기 위한 노력을 국가와 시민의 차원에...


In [61]:
school.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1366 entries, 0 to 1365
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   학교명     1366 non-null   object 
 1   위치      1366 non-null   object 
 2   학생수     1354 non-null   float64
 3   예산(만원)  1240 non-null   float64
 4   매칭교과    1366 non-null   object 
 5   강좌명     1366 non-null   object 
 6   활동목표    1366 non-null   object 
 7   위도      1366 non-null   float64
 8   경도      1366 non-null   float64
 9   제목_활동   1366 non-null   object 
dtypes: float64(4), object(6)
memory usage: 106.8+ KB


In [64]:
program = pd.read_csv(f'{ROOT}/data/프로그램_통합.csv', encoding= 'cp949')
program.head()

,제목,인원,참가비,기관명,세부정보,주소,위도,경도
0,"""DIY"" 자양 바로담아 고추장담그기",1000,"30,000원",자양전통장,고추장 담금 활동을 통해 요리에 대한 흥미를 발견하고 일의 중요성을 알아본다 자신이...,경북 영천시 자양면 별빛로 961-14,36.138262,129.007378
1,"""DIY"" 자양 바로담아 고추장담그기(실속)",999,"25,000원",자양전통장,전통 고추장을 직접 담아 안정한 먹거리를 먹어보고 우리 전통 고추장을 쉽게 담아볼 ...,경북 영천시 자양면 별빛로 961-14,36.138262,129.007378
2,가상현실전문가,10,무료,문경시청소년문화의집,(행복한 진로) 가상현실전문가,경상북도 문경시 모전동 77-13,36.591216,128.187087
3,목공예원,10,무료,문경시청소년문화의집,(행복한 진로) 목공예원,경상북도 문경시 모전동 77-13,36.591216,128.187087
4,애견훈련사,10,무료,문경시청소년문화의집,(행복한 진로) 애견훈련사,경상북도 문경시 모전동 77-13,36.591216,128.187087


In [67]:
program['세부정보'].fillna("", inplace=True)
program['제목_활동'] = program['제목']
program.head()

,제목,인원,참가비,기관명,세부정보,주소,위도,경도,제목_활동
0,"""DIY"" 자양 바로담아 고추장담그기",1000,"30,000원",자양전통장,고추장 담금 활동을 통해 요리에 대한 흥미를 발견하고 일의 중요성을 알아본다 자신이...,경북 영천시 자양면 별빛로 961-14,36.138262,129.007378,"""DIY"" 자양 바로담아 고추장담그기"
1,"""DIY"" 자양 바로담아 고추장담그기(실속)",999,"25,000원",자양전통장,전통 고추장을 직접 담아 안정한 먹거리를 먹어보고 우리 전통 고추장을 쉽게 담아볼 ...,경북 영천시 자양면 별빛로 961-14,36.138262,129.007378,"""DIY"" 자양 바로담아 고추장담그기(실속)"
2,가상현실전문가,10,무료,문경시청소년문화의집,(행복한 진로) 가상현실전문가,경상북도 문경시 모전동 77-13,36.591216,128.187087,가상현실전문가
3,목공예원,10,무료,문경시청소년문화의집,(행복한 진로) 목공예원,경상북도 문경시 모전동 77-13,36.591216,128.187087,목공예원
4,애견훈련사,10,무료,문경시청소년문화의집,(행복한 진로) 애견훈련사,경상북도 문경시 모전동 77-13,36.591216,128.187087,애견훈련사


In [68]:
program.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 358 entries, 0 to 357
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   제목      358 non-null    object 
 1   인원      358 non-null    int64  
 2   참가비     358 non-null    object 
 3   기관명     358 non-null    object 
 4   세부정보    358 non-null    object 
 5   주소      357 non-null    object 
 6   위도      358 non-null    float64
 7   경도      358 non-null    float64
 8   제목_활동   358 non-null    object 
dtypes: float64(2), int64(1), object(6)
memory usage: 25.3+ KB


### 2. Text Preprocessing(removing stopwords)

In [69]:
def clean_text(text):
    text = text.replace(".", "").strip()
    text = text.replace("·", " ").strip()
    pattern = '[^ ㄱ-ㅣ가-힣|]+'
    text = re.sub(pattern=pattern, repl='', string=text)
    return text

In [70]:
school['제목_활동'] = school['제목_활동'].apply(clean_text)
program['제목_활동'] = program['제목_활동'].apply(clean_text)

### 3. Tokenization(Mecab)

In [71]:
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

def get_nouns(tokenizer, lemmatizer, sentence):
    tagged = tokenizer.pos(sentence)
    nouns = [lemmatizer.lemmatize(s, pos='n') for s, t in tagged if t in ['NNG', 'NNP', 'VA', 'XR']]
    return nouns

def tokenize(df):
    tokenizer = MeCab()
    lemmatizer = WordNetLemmatizer()
    processed_data = []
    for sent in tqdm(df['제목_활동']):
        sentence = clean_text(str(sent).replace("\n", "").strip())
        processed_data.append(get_nouns(tokenizer, lemmatizer, sentence))
    return processed_data

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [72]:
school['tokenized'] = tokenize(school)
school.head()

100%|██████████| 1366/1366 [00:00<00:00, 1701.28it/s]


,학교명,위치,학생수,예산(만원),매칭교과,강좌명,활동목표,위도,경도,제목_활동,tokenized
0,경북체육중학교,경산시,26.0,700.0,과학,내가 닮고 싶은 과학자 이야기,1. 과학자가 들려주는 과학 이야기를 읽고 내가 닮고 싶은 과학자에 대한 자료를 ...,35.812226,128.738053,내가 닮고 싶은 과학자 이야기 과학자가 들려주는 과학 이야기를 읽고 내가 닮고 싶...,"[과학자, 이야기, 과학자, 과학, 이야기, 과학자, 자료, 조사, 분석, 정리, ..."
1,경북체육중학교,경산시,26.0,700.0,국어,문학에 빠지다,"1. 문학에 대한 이해를 높이고, 책 읽기를 통하여 문학에 대한 이해를 기를 수 ...",35.812226,128.738053,문학에 빠지다 문학에 대한 이해를 높이고 책 읽기를 통하여 문학에 대한 이해를 기...,"[문학, 문학, 이해, 책, 문학, 이해, 독후, 다양, 활동, 표현, 문학, 감수]"
2,경북체육중학교,경산시,26.0,700.0,사회,영화 속 사회,"1. 일본군 위안부와 관련된 자료를 찾아 그 실상을 알고, 홍보 굿즈를 제작해 보...",35.812226,128.738053,영화 속 사회 일본군 위안부와 관련된 자료를 찾아 그 실상을 알고 홍보 굿즈를 제...,"[영화, 속, 사회, 일본군, 위안부, 관련, 자료, 실상, 홍보, 굿, 즈, 제작..."
3,경산여자중학교,경산시,281.0,NaN,국어,시 감상과 모방시 창작,1. 문학은 심미적 체험을 바탕으로 한 다양한 소통 활동임을 알고 문학 활동을 할...,35.817764,128.729945,시 감상과 모방시 창작 문학은 심미적 체험을 바탕으로 한 다양한 소통 활동임을 알...,"[시, 감상, 모방, 창작, 문학, 심미, 체험, 바탕, 다양, 소통, 활동, 문학..."
4,경산여자중학교,경산시,281.0,NaN,사회,내가 만들어가는 민주정치,1. 민주 정치를 실현하기 위한 노력을 국가와 시민의 차원에서 탐구하여 설명할 수 ...,35.817764,128.729945,내가 만들어가는 민주정치 민주 정치를 실현하기 위한 노력을 국가와 시민의 차원에서 ...,"[민주, 정치, 민주, 정치, 실현, 노력, 국가, 시민, 차원, 탐구, 설명, 다..."


In [73]:
print(school.loc[3,'제목_활동'])
print(school.loc[3,'tokenized'])

시 감상과 모방시 창작  문학은 심미적 체험을 바탕으로 한 다양한 소통 활동임을 알고 문학 활동을 할 수 있다   시 작품을 감상 한 후 가장 감명받은 작품을 선정 한 후 작품을 기초로 한 편의 모방시를 창작한다  모둠화 그리기 및 신문 스크랩만들기
['시', '감상', '모방', '창작', '문학', '심미', '체험', '바탕', '다양', '소통', '활동', '문학', '활동', '시', '작품', '감상', '후', '감명', '작품', '선정', '후', '작품', '기초', '모방', '시', '창작', '모둠', '신문', '스크랩']


In [74]:
school['tokenized']

0       [과학자, 이야기, 과학자, 과학, 이야기, 과학자, 자료, 조사, 분석, 정리, ...
1         [문학, 문학, 이해, 책, 문학, 이해, 독후, 다양, 활동, 표현, 문학, 감수]
2       [영화, 속, 사회, 일본군, 위안부, 관련, 자료, 실상, 홍보, 굿, 즈, 제작...
3       [시, 감상, 모방, 창작, 문학, 심미, 체험, 바탕, 다양, 소통, 활동, 문학...
4       [민주, 정치, 민주, 정치, 실현, 노력, 국가, 시민, 차원, 탐구, 설명, 다...
                              ...                        
1361    [과학, 용어, 사전, 과학, 용어, 사전, 활동, 과학, 기초, 용어, 이해, 자...
1362    [창의력, 나만, 과학, 책, 과학, 책, 활동, 과학, 기초, 지식, 이해, 자신...
1363    [로봇, 코딩, 로봇, 구조, 센서, 기능, 이해, 코딩, 로봇, 동작, 기능, 제...
1364    [키즈, 국가, 문화, 차, 문화, 정보, 수용, 이해, 태도, 영어, 정보, 문해...
1365    [수퍼, 퍼, 국제, 사회, 문화, 이해, 문화, 이해, 국제, 사회, 이해, 능력...
Name: tokenized, Length: 1366, dtype: object

In [75]:
program['tokenized'] = tokenize(program)
program.head()

100%|██████████| 358/358 [00:00<00:00, 7618.73it/s]


,제목,인원,참가비,기관명,세부정보,주소,위도,경도,제목_활동,tokenized
0,"""DIY"" 자양 바로담아 고추장담그기",1000,"30,000원",자양전통장,고추장 담금 활동을 통해 요리에 대한 흥미를 발견하고 일의 중요성을 알아본다 자신이...,경북 영천시 자양면 별빛로 961-14,36.138262,129.007378,자양 바로담아 고추장담그기,"[자양, 고추장]"
1,"""DIY"" 자양 바로담아 고추장담그기(실속)",999,"25,000원",자양전통장,전통 고추장을 직접 담아 안정한 먹거리를 먹어보고 우리 전통 고추장을 쉽게 담아볼 ...,경북 영천시 자양면 별빛로 961-14,36.138262,129.007378,자양 바로담아 고추장담그기실속,"[자양, 고추장, 실속]"
2,가상현실전문가,10,무료,문경시청소년문화의집,(행복한 진로) 가상현실전문가,경상북도 문경시 모전동 77-13,36.591216,128.187087,가상현실전문가,"[가상현실, 전문가]"
3,목공예원,10,무료,문경시청소년문화의집,(행복한 진로) 목공예원,경상북도 문경시 모전동 77-13,36.591216,128.187087,목공예원,"[목공, 예원]"
4,애견훈련사,10,무료,문경시청소년문화의집,(행복한 진로) 애견훈련사,경상북도 문경시 모전동 77-13,36.591216,128.187087,애견훈련사,"[애견, 훈련, 사]"


In [76]:
school_token = school['tokenized'].tolist()

merged_school = [item for sublist in school_token for item in sublist]

print(merged_school)

['과학자', '이야기', '과학자', '과학', '이야기', '과학자', '자료', '조사', '분석', '정리', '과학', '책', '자신', '과학', '책', '영상', '친구', '소개', '상평', '활동', '과학', '지식', '해석', '토론', '문학', '문학', '이해', '책', '문학', '이해', '독후', '다양', '활동', '표현', '문학', '감수', '영화', '속', '사회', '일본군', '위안부', '관련', '자료', '실상', '홍보', '굿', '즈', '제작', '역할', '책임', '일본군', '위안부', '실상', '세계', '홍보', '계획', '후', '홍보', '나라', '랜드마크', '스티커', '제작물', '시', '감상', '모방', '창작', '문학', '심미', '체험', '바탕', '다양', '소통', '활동', '문학', '활동', '시', '작품', '감상', '후', '감명', '작품', '선정', '후', '작품', '기초', '모방', '시', '창작', '모둠', '신문', '스크랩', '민주', '정치', '민주', '정치', '실현', '노력', '국가', '시민', '차원', '탐구', '설명', '다양', '가치', '이익', '정치', '과정', '조정', '정치', '과정', '참여', '정치', '주체', '역할', '설명', '통계', '환경', '교육', '자료', '홍수', '속', '현대', '사회', '다양', '자료', '정보', '수집', '정리', '분석', '활용', '정보', '해석', '비판', '평가', '의사소통', '자료', '의미', '해석', '능력', '통계', '환경', '보존', '필요', '인식', '지구', '생태', '환경', '인간', '환경', '영향', '이해', '인간', '환경', '파괴', '설명', '환경', '파괴', '결과', '인간', '영향', '영어', '발음', '영어', '동화', '제작', '인권', 

In [77]:
# 제거할 단어 5개 = 다양, 활동, 능력, 이해, 문제

from collections import Counter
t1 = Counter(merged_school)
print(t1.most_common())

[('수학', 773), ('다양', 697), ('과학', 638), ('활동', 592), ('능력', 548), ('영어', 456), ('이해', 427), ('생활', 373), ('탐구', 340), ('자신', 297), ('문제', 291), ('속', 272), ('표현', 271), ('함양', 231), ('흥미', 231), ('세계', 220), ('체험', 214), ('사회', 212), ('활용', 205), ('문화', 196), ('반', 195), ('창의', 193), ('해결', 188), ('의사소통', 164), ('관련', 162), ('문학', 161), ('책', 154), ('학습', 154), ('원리', 154), ('환경', 144), ('과정', 138), ('사고력', 128), ('독서', 127), ('삶', 126), ('향상', 126), ('제작', 124), ('작품', 124), ('역량', 123), ('학생', 121), ('생각', 120), ('사고', 117), ('주제', 110), ('시', 108), ('일상', 104), ('실생활', 104), ('세상', 102), ('경험', 96), ('여행', 95), ('이야기', 92), ('감상', 91), ('방법', 91), ('관심', 89), ('글', 87), ('태도', 86), ('매체', 84), ('현상', 82), ('기술', 80), ('창의력', 79), ('자연', 79), ('바탕', 78), ('실험', 78), ('실천', 78), ('자료', 76), ('필요', 76), ('사용', 76), ('내용', 71), ('지역', 71), ('토론', 70), ('말', 70), ('인식', 69), ('미래', 69), ('설명', 68), ('정보', 67), ('지식', 66), ('나라', 65), ('시민', 65), ('협업', 65), ('파악', 61), ('미디어', 60), ('개념'

In [78]:
program_token = program['tokenized'].tolist()

merged_program = [item for sublist in program_token for item in sublist]

print(merged_program)

['자양', '고추장', '자양', '고추장', '실속', '가상현실', '전문가', '목공', '예원', '애견', '훈련', '사', '플로', '드', '코딩', '융합', '전문가', '청년', '농부', '카지노', '딜러', '환경', '활동가', '웹툰', '작가', '드론', '조종사', '디저트', '디자이너', '바리스타', '전문가', '핸드', '드립', '전통', '장류', '과일', '와인', '과일', '식초', '체험', '위대', '함성', '청소년', '순국', '선열', '함성', '독립', '역사', '교육', '힐링', '통기타', '요리', '체험', '반', '지역', '자원', '연계', '농촌', '체험', '관광지', '참외', '피클', '바베큐', '팜', '파티', '피크닉', '도시락', '모델링', '전문가', '중학생', '미래', '기출', '체험', '모델링', '체험', '아트', '펜', '조형물', '제작', '창', '메이커', '스마트', '펜', '창작물', '제작', '펜', '기초', '반', '펜', '디자인', '전문가', '중학생', '펜', '창', '체험', '프린터', '전문가', '중학생', '프린트', '체험', '프린팅', '콘텐츠', '크리에이터', '중학생', '펜', '창', '체험', '차', '산업', '혁명', '시대', '진로', '창시자', '창', '직진', '프로그램', '컷', '그림일기', '차', '산업', '체험', '생강', '지역', '농산물', '활용', '창', '과학자', '주행', '자동차', '프로그램', '활용', '건축', '디자인', '체험', '프로그램', '원목', '목공', '체험', '선반', '책꽂이', '연필꽂이', '스토리', '박스', '코티지', '박스', '반달꽂이', '드림', '주왕산', '국립', '공원', '직업', '체험', '프로그램', '동식물', '전문가', '환경', '교육', '전문가', '드

In [80]:
# 제거할 단어 5개 = 체험, 진로, 중학생, 미래, 직업

from collections import Counter
t2 = Counter(merged_program)
print(t2.most_common())

[('체험', 128), ('진로', 28), ('중학생', 27), ('공예', 27), ('미래', 21), ('직업', 21), ('전문가', 19), ('교육', 19), ('창', 18), ('아트', 15), ('캠프', 14), ('메이커', 11), ('빵', 11), ('융합', 10), ('와인', 9), ('제작', 9), ('프로그램', 9), ('도전', 9), ('교실', 9), ('전통', 8), ('청소년', 8), ('디자인', 8), ('드론', 7), ('바리스타', 7), ('펜', 7), ('활용', 7), ('과학', 7), ('과일', 6), ('역사', 6), ('콘텐츠', 6), ('산업', 6), ('클래스', 6), ('댄스', 6), ('컵', 6), ('레진', 6), ('예술', 6), ('과제', 6), ('고추장', 5), ('플로', 5), ('코딩', 5), ('환경', 5), ('반', 5), ('파티', 5), ('차', 5), ('자동차', 5), ('소개', 5), ('테라', 5), ('농장', 5), ('케익', 5), ('디자이너', 4), ('요리', 4), ('농촌', 4), ('기초', 4), ('박스', 4), ('비누', 4), ('기술', 4), ('가죽', 4), ('건강', 4), ('버드', 4), ('금융', 4), ('방송', 4), ('메타', 4), ('버스', 4), ('캘리', 4), ('그라피', 4), ('내손', 4), ('네일', 4), ('도자기', 4), ('학습', 4), ('딸기', 4), ('오르골', 4), ('개발자', 4), ('쇼콜라티에', 4), ('압', 4), ('플라워', 4), ('오디', 4), ('원예', 4), ('리스트', 4), ('케이크', 4), ('증강현실', 4), ('쿠키', 4), ('가상현실', 3), ('활동가', 3), ('작가', 3), ('조종사', 3), ('핸드', 3), ('식초', 3), ('통

### 4. Modeling

#### Method 1 : LDA Topic Modeling

In [81]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.metrics.pairwise import cosine_similarity

In [82]:
combined_tokens = list(school['tokenized']) + list(program['tokenized'])
combined_tokens = [' '.join(tokens) for tokens in combined_tokens]

vectorizer = CountVectorizer()
vectors = vectorizer.fit_transform(combined_tokens)

lda_model = LatentDirichletAllocation(n_components=10, random_state=42)
lda_vectors = lda_model.fit_transform(vectors)

lda_similarity_matrix = cosine_similarity(lda_vectors[:len(school)], lda_vectors[len(school):])

In [83]:
lda_similarity_matrix.shape

(1366, 358)

In [84]:
max_indexes = np.argmax(lda_similarity_matrix, axis=1)

In [85]:
result_lda = school.copy()

In [87]:
result_lda['match_program'] = ""
result_lda['match_center'] = ""

for i in range(len(result_lda)):
  program_index = max_indexes[i]
  result_lda['match_program'][i] = program['제목'][program_index]
  result_lda['match_center'][i] = program['기관명'][program_index]

<ipython-input-87-e810b33921bb>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_lda['match_program'][i] = program['제목'][program_index]
<ipython-input-87-e810b33921bb>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_lda['match_center'][i] = program['기관명'][program_index]


In [88]:
result_lda

,학교명,위치,학생수,예산(만원),매칭교과,강좌명,활동목표,위도,경도,제목_활동,tokenized,match_program,match_center
0,경북체육중학교,경산시,26.0,700.0,과학,내가 닮고 싶은 과학자 이야기,1. 과학자가 들려주는 과학 이야기를 읽고 내가 닮고 싶은 과학자에 대한 자료를 ...,35.812226,128.738053,내가 닮고 싶은 과학자 이야기 과학자가 들려주는 과학 이야기를 읽고 내가 닮고 싶...,"[과학자, 이야기, 과학자, 과학, 이야기, 과학자, 자료, 조사, 분석, 정리, ...",과학개념 완성 실험과학,상주시청소년수련관
1,경북체육중학교,경산시,26.0,700.0,국어,문학에 빠지다,"1. 문학에 대한 이해를 높이고, 책 읽기를 통하여 문학에 대한 이해를 기를 수 ...",35.812226,128.738053,문학에 빠지다 문학에 대한 이해를 높이고 책 읽기를 통하여 문학에 대한 이해를 기...,"[문학, 문학, 이해, 책, 문학, 이해, 독후, 다양, 활동, 표현, 문학, 감수]",예술로 마음을 도닥인다 : 도닥도닥 사회적 협동조합,도닥도닥 사회적 협동조합
2,경북체육중학교,경산시,26.0,700.0,사회,영화 속 사회,"1. 일본군 위안부와 관련된 자료를 찾아 그 실상을 알고, 홍보 굿즈를 제작해 보...",35.812226,128.738053,영화 속 사회 일본군 위안부와 관련된 자료를 찾아 그 실상을 알고 홍보 굿즈를 제...,"[영화, 속, 사회, 일본군, 위안부, 관련, 자료, 실상, 홍보, 굿, 즈, 제작...",메디컬(해부학) 멸치,교육사회적기업 포인트(경북북부센터)
3,경산여자중학교,경산시,281.0,NaN,국어,시 감상과 모방시 창작,1. 문학은 심미적 체험을 바탕으로 한 다양한 소통 활동임을 알고 문학 활동을 할...,35.817764,128.729945,시 감상과 모방시 창작 문학은 심미적 체험을 바탕으로 한 다양한 소통 활동임을 알...,"[시, 감상, 모방, 창작, 문학, 심미, 체험, 바탕, 다양, 소통, 활동, 문학...",영어로배우는체육교실,라온스포츠복지사회적협동조합
4,경산여자중학교,경산시,281.0,NaN,사회,내가 만들어가는 민주정치,1. 민주 정치를 실현하기 위한 노력을 국가와 시민의 차원에서 탐구하여 설명할 수 ...,35.817764,128.729945,내가 만들어가는 민주정치 민주 정치를 실현하기 위한 노력을 국가와 시민의 차원에서 ...,"[민주, 정치, 민주, 정치, 실현, 노력, 국가, 시민, 차원, 탐구, 설명, 다...","특성화고 학과체험(금융정보, 마케팅물류, 인터넷 비지니스 학과 체험)",명인정보고등학교
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1361,흥해중학교,포항시,151.0,1700.0,과학,나만의 과학용어 사전 만들기,1. 과학용어 사전 만들기 활동을 통해 과학의 기초용어를 이해할 수 있다.\n2. ...,36.109218,129.341171,나만의 과학용어 사전 만들기 과학용어 사전 만들기 활동을 통해 과학의 기초용어를 이...,"[과학, 용어, 사전, 과학, 용어, 사전, 활동, 과학, 기초, 용어, 이해, 자...","‘위대한 함성’ 청소년, 순국 선열의 함성을 외치다.(독립역사 교육)",안동청소년문화센터
1362,흥해중학교,포항시,151.0,1700.0,과학,창의력 넘치는 나만의 과학책,1. 과학책 만들기 활동을 통해 과학의 기초지식을 이해할 수 있다.\n2. 자신의 ...,36.109218,129.341171,창의력 넘치는 나만의 과학책 과학책 만들기 활동을 통해 과학의 기초지식을 이해할 수...,"[창의력, 나만, 과학, 책, 과학, 책, 활동, 과학, 기초, 지식, 이해, 자신...",과학창작 VR캠프,경산시계림청소년수련원
1363,흥해중학교,포항시,151.0,1700.0,정보,로봇 코딩,1. 로봇의 구조와 각 센서의 기능을 이해하고 코딩할 수 있다.\n2. 로봇의 여러...,36.109218,129.341171,로봇 코딩 로봇의 구조와 각 센서의 기능을 이해하고 코딩할 수 있다 로봇의 여러 가...,"[로봇, 코딩, 로봇, 구조, 센서, 기능, 이해, 코딩, 로봇, 동작, 기능, 제...",드림콘서트(2) 진로학습법캠프[진로캠프 & 자기주도학습법캠프] - Dream Con...,글로벌인재개발연구원
1364,흥해중학교,포항시,151.0,1700.0,영어,퍼니 멍키즈,1. 국가별 문화 차이나 다문화에 대한 정보를 수용하고 이해하는 태도를 기른다.\n...,36.109218,129.341171,퍼니 멍키즈 국가별 문화 차이나 다문화에 대한 정보를 수용하고 이해하는 태도를 기른...,"[키즈, 국가, 문화, 차, 문화, 정보, 수용, 이해, 태도, 영어, 정보, 문해...",청소년합창단 음악캠프,경주시청소년수련관


In [89]:
result_lda[['학교명', '강좌명','match_center', 'match_program']].iloc[[0,1,2,44,1361,1363],:]

,학교명,강좌명,match_center,match_program
0,경북체육중학교,내가 닮고 싶은 과학자 이야기,상주시청소년수련관,과학개념 완성 실험과학
1,경북체육중학교,문학에 빠지다,도닥도닥 사회적 협동조합,예술로 마음을 도닥인다 : 도닥도닥 사회적 협동조합
2,경북체육중학교,영화 속 사회,교육사회적기업 포인트(경북북부센터),메디컬(해부학) 멸치
44,삼성현중학교,영어 잡지 만들기,경주시청소년수련관,청소년합창단 음악캠프
1361,흥해중학교,나만의 과학용어 사전 만들기,안동청소년문화센터,"‘위대한 함성’ 청소년, 순국 선열의 함성을 외치다.(독립역사 교육)"
1363,흥해중학교,로봇 코딩,글로벌인재개발연구원,드림콘서트(2) 진로학습법캠프[진로캠프 & 자기주도학습법캠프] - Dream Con...


#### Method 1'(5 words elimination) : LDA Topic Modeling

In [90]:
school_1 = school.copy()

In [91]:
# 제거할 단어
words_to_remove = ['다양', '활동', '능력', '이해', '문제']


# 단어 제거
school_1['tokenized'] = school_1['tokenized'].apply(lambda x: [word for word in x if word not in words_to_remove])

# 결과 출력
school_1.head()

,학교명,위치,학생수,예산(만원),매칭교과,강좌명,활동목표,위도,경도,제목_활동,tokenized
0,경북체육중학교,경산시,26.0,700.0,과학,내가 닮고 싶은 과학자 이야기,1. 과학자가 들려주는 과학 이야기를 읽고 내가 닮고 싶은 과학자에 대한 자료를 ...,35.812226,128.738053,내가 닮고 싶은 과학자 이야기 과학자가 들려주는 과학 이야기를 읽고 내가 닮고 싶...,"[과학자, 이야기, 과학자, 과학, 이야기, 과학자, 자료, 조사, 분석, 정리, ..."
1,경북체육중학교,경산시,26.0,700.0,국어,문학에 빠지다,"1. 문학에 대한 이해를 높이고, 책 읽기를 통하여 문학에 대한 이해를 기를 수 ...",35.812226,128.738053,문학에 빠지다 문학에 대한 이해를 높이고 책 읽기를 통하여 문학에 대한 이해를 기...,"[문학, 문학, 책, 문학, 독후, 표현, 문학, 감수]"
2,경북체육중학교,경산시,26.0,700.0,사회,영화 속 사회,"1. 일본군 위안부와 관련된 자료를 찾아 그 실상을 알고, 홍보 굿즈를 제작해 보...",35.812226,128.738053,영화 속 사회 일본군 위안부와 관련된 자료를 찾아 그 실상을 알고 홍보 굿즈를 제...,"[영화, 속, 사회, 일본군, 위안부, 관련, 자료, 실상, 홍보, 굿, 즈, 제작..."
3,경산여자중학교,경산시,281.0,NaN,국어,시 감상과 모방시 창작,1. 문학은 심미적 체험을 바탕으로 한 다양한 소통 활동임을 알고 문학 활동을 할...,35.817764,128.729945,시 감상과 모방시 창작 문학은 심미적 체험을 바탕으로 한 다양한 소통 활동임을 알...,"[시, 감상, 모방, 창작, 문학, 심미, 체험, 바탕, 소통, 문학, 시, 작품,..."
4,경산여자중학교,경산시,281.0,NaN,사회,내가 만들어가는 민주정치,1. 민주 정치를 실현하기 위한 노력을 국가와 시민의 차원에서 탐구하여 설명할 수 ...,35.817764,128.729945,내가 만들어가는 민주정치 민주 정치를 실현하기 위한 노력을 국가와 시민의 차원에서 ...,"[민주, 정치, 민주, 정치, 실현, 노력, 국가, 시민, 차원, 탐구, 설명, 가..."


In [92]:
program_1 = program.copy()

In [93]:
# 제거할 단어
words_to_remove = ['체험', '진로', '중학생', '미래', '직업']

# 단어 제거
program_1['tokenized'] = program_1['tokenized'].apply(lambda x: [word for word in x if word not in words_to_remove])

# 결과 출력
program_1.head()

,제목,인원,참가비,기관명,세부정보,주소,위도,경도,제목_활동,tokenized
0,"""DIY"" 자양 바로담아 고추장담그기",1000,"30,000원",자양전통장,고추장 담금 활동을 통해 요리에 대한 흥미를 발견하고 일의 중요성을 알아본다 자신이...,경북 영천시 자양면 별빛로 961-14,36.138262,129.007378,자양 바로담아 고추장담그기,"[자양, 고추장]"
1,"""DIY"" 자양 바로담아 고추장담그기(실속)",999,"25,000원",자양전통장,전통 고추장을 직접 담아 안정한 먹거리를 먹어보고 우리 전통 고추장을 쉽게 담아볼 ...,경북 영천시 자양면 별빛로 961-14,36.138262,129.007378,자양 바로담아 고추장담그기실속,"[자양, 고추장, 실속]"
2,가상현실전문가,10,무료,문경시청소년문화의집,(행복한 진로) 가상현실전문가,경상북도 문경시 모전동 77-13,36.591216,128.187087,가상현실전문가,"[가상현실, 전문가]"
3,목공예원,10,무료,문경시청소년문화의집,(행복한 진로) 목공예원,경상북도 문경시 모전동 77-13,36.591216,128.187087,목공예원,"[목공, 예원]"
4,애견훈련사,10,무료,문경시청소년문화의집,(행복한 진로) 애견훈련사,경상북도 문경시 모전동 77-13,36.591216,128.187087,애견훈련사,"[애견, 훈련, 사]"


In [94]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.metrics.pairwise import cosine_similarity

In [95]:
combined_tokens_1 = list(school_1['tokenized']) + list(program_1['tokenized'])
combined_tokens_1 = [' '.join(tokens) for tokens in combined_tokens_1]

vectorizer = CountVectorizer()
vectors_1 = vectorizer.fit_transform(combined_tokens_1)

lda_model_1 = LatentDirichletAllocation(n_components=8, random_state=42)
lda_vectors_1 = lda_model_1.fit_transform(vectors_1)

lda_similarity_matrix_1 = cosine_similarity(lda_vectors_1[:len(school)], lda_vectors_1[len(school):])

In [96]:
lda_similarity_matrix_1.shape

(1366, 358)

In [97]:
max_indexes_1 = np.argmax(lda_similarity_matrix_1, axis=1)

In [98]:
result_lda_1 = school.copy()

In [101]:
result_lda_1['matched_program'] = ""
result_lda_1['matched_center'] = ""
for i in range(len(result_lda_1)):
  program_index_1 = max_indexes_1[i]
  result_lda_1['matched_center'][i] = program['기관명'][program_index_1]
  result_lda_1['matched_program'][i] = program['제목'][program_index_1]

<ipython-input-101-c96892e36e3a>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_lda_1['matched_center'][i] = program['기관명'][program_index_1]
<ipython-input-101-c96892e36e3a>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_lda_1['matched_program'][i] = program['제목'][program_index_1]


In [110]:
result_lda_1[['학교명', '강좌명','matched_center', 'matched_program']].iloc[[20, 112, 114, 159, 168, 177],:]

,학교명,강좌명,matched_center,matched_program
20,경산중학교,인공지능 시대 하이테크 세상,교육사회적기업 포인트(경북북부센터),과학수사 KCSI / 과학수사대는 무슨 일을 할까?
112,경주중학교,영어로 읽는 과학 세상,문경시청소년문화의집,Let’s sing in English 영어 노래 부르기
114,경주중학교,스포츠 국제화,청도곤충나라,곤충 사육 체험
159,서라벌여자중학교,즐기는 체험 수학반,핸즈체험터,"우드버닝 명패, 필통, 시계"
168,선덕여자중학교,스마트한 미래를 프로그래밍하다,(주)스카이엘앤에이치,메타버스 학교 개발자
177,신라중학교,과학과 환경,농업회사법인(주)한국와인,"뭐야! 와인이 과학이라고!? 99.9%과일만으로 만들어지는 와인, 어떻게 만들어질까?"


#### Method 2 : TF-IDF Cosine Similarity

In [111]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [112]:
combined_tokens = list(school['tokenized']) + list(program['tokenized'])
combined_tokens = [' '.join(tokens) for tokens in combined_tokens]

vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(combined_tokens)

tfidf_similarity_matrix = cosine_similarity(vectors[:len(school)], vectors[len(school):])

In [113]:
tfidf_similarity_matrix.shape

(1366, 358)

In [114]:
max_indexes = np.argmax(tfidf_similarity_matrix, axis=1)

In [115]:
result_tfidf = school.copy()

In [116]:
result_tfidf['match_program'] = ""

for i in range(len(result_tfidf)):
  program_index = max_indexes[i]
  result_tfidf['match_program'][i] = program['제목'][program_index]

<ipython-input-116-dc288aa0f388>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_tfidf['match_program'][i] = program['제목'][program_index]


In [125]:
result_tfidf

,학교명,위치,학생수,예산(만원),매칭교과,강좌명,활동목표,위도,경도,제목_활동,tokenized,match_program
0,경북체육중학교,경산시,26.0,700.0,과학,내가 닮고 싶은 과학자 이야기,1. 과학자가 들려주는 과학 이야기를 읽고 내가 닮고 싶은 과학자에 대한 자료를 ...,35.812226,128.738053,내가 닮고 싶은 과학자 이야기 과학자가 들려주는 과학 이야기를 읽고 내가 닮고 싶...,"[과학자, 이야기, 과학자, 과학, 이야기, 과학자, 자료, 조사, 분석, 정리, ...","AI 창의과학(자율주행 자동차, 호버볼 만들기)"
1,경북체육중학교,경산시,26.0,700.0,국어,문학에 빠지다,"1. 문학에 대한 이해를 높이고, 책 읽기를 통하여 문학에 대한 이해를 기를 수 ...",35.812226,128.738053,문학에 빠지다 문학에 대한 이해를 높이고 책 읽기를 통하여 문학에 대한 이해를 기...,"[문학, 문학, 이해, 책, 문학, 이해, 독후, 다양, 활동, 표현, 문학, 감수]",해양체험활동
2,경북체육중학교,경산시,26.0,700.0,사회,영화 속 사회,"1. 일본군 위안부와 관련된 자료를 찾아 그 실상을 알고, 홍보 굿즈를 제작해 보...",35.812226,128.738053,영화 속 사회 일본군 위안부와 관련된 자료를 찾아 그 실상을 알고 홍보 굿즈를 제...,"[영화, 속, 사회, 일본군, 위안부, 관련, 자료, 실상, 홍보, 굿, 즈, 제작...",사회적기업가의 삶
3,경산여자중학교,경산시,281.0,NaN,국어,시 감상과 모방시 창작,1. 문학은 심미적 체험을 바탕으로 한 다양한 소통 활동임을 알고 문학 활동을 할...,35.817764,128.729945,시 감상과 모방시 창작 문학은 심미적 체험을 바탕으로 한 다양한 소통 활동임을 알...,"[시, 감상, 모방, 창작, 문학, 심미, 체험, 바탕, 다양, 소통, 활동, 문학...",창작이 즐거운 코딩
4,경산여자중학교,경산시,281.0,NaN,사회,내가 만들어가는 민주정치,1. 민주 정치를 실현하기 위한 노력을 국가와 시민의 차원에서 탐구하여 설명할 수 ...,35.817764,128.729945,내가 만들어가는 민주정치 민주 정치를 실현하기 위한 노력을 국가와 시민의 차원에서 ...,"[민주, 정치, 민주, 정치, 실현, 노력, 국가, 시민, 차원, 탐구, 설명, 다...",찾아가는 전문직업인과의 만남(다양한 진로체험)
...,...,...,...,...,...,...,...,...,...,...,...,...
1361,흥해중학교,포항시,151.0,1700.0,과학,나만의 과학용어 사전 만들기,1. 과학용어 사전 만들기 활동을 통해 과학의 기초용어를 이해할 수 있다.\n2. ...,36.109218,129.341171,나만의 과학용어 사전 만들기 과학용어 사전 만들기 활동을 통해 과학의 기초용어를 이...,"[과학, 용어, 사전, 과학, 용어, 사전, 활동, 과학, 기초, 용어, 이해, 자...",과학개념 완성 실험과학
1362,흥해중학교,포항시,151.0,1700.0,과학,창의력 넘치는 나만의 과학책,1. 과학책 만들기 활동을 통해 과학의 기초지식을 이해할 수 있다.\n2. 자신의 ...,36.109218,129.341171,창의력 넘치는 나만의 과학책 과학책 만들기 활동을 통해 과학의 기초지식을 이해할 수...,"[창의력, 나만, 과학, 책, 과학, 책, 활동, 과학, 기초, 지식, 이해, 자신...",과학개념 완성 실험과학
1363,흥해중학교,포항시,151.0,1700.0,정보,로봇 코딩,1. 로봇의 구조와 각 센서의 기능을 이해하고 코딩할 수 있다.\n2. 로봇의 여러...,36.109218,129.341171,로봇 코딩 로봇의 구조와 각 센서의 기능을 이해하고 코딩할 수 있다 로봇의 여러 가...,"[로봇, 코딩, 로봇, 구조, 센서, 기능, 이해, 코딩, 로봇, 동작, 기능, 제...",로봇공학자(중학생을 위한 창의융합 미래로봇 체험)
1364,흥해중학교,포항시,151.0,1700.0,영어,퍼니 멍키즈,1. 국가별 문화 차이나 다문화에 대한 정보를 수용하고 이해하는 태도를 기른다.\n...,36.109218,129.341171,퍼니 멍키즈 국가별 문화 차이나 다문화에 대한 정보를 수용하고 이해하는 태도를 기른...,"[키즈, 국가, 문화, 차, 문화, 정보, 수용, 이해, 태도, 영어, 정보, 문해...",3D펜 기초반


In [118]:
result_tfidf['match_program'].value_counts()

과학개념 완성 실험과학                        178
Let’s sing in English 영어 노래 부르기     111
해양체험활동                               47
사회적기업가의 삶                            45
기후행동가(환경 관련 직업 체험)                   35
                                   ... 
우리 동네  해결사 청소년 마을탐사대                  1
냅킨아트 ~ 패브릭 천가방 만들기                    1
증강현실 전문가(중학생을 위한 미래신기술체험_AR증강현실)      1
부채냅킨공예                                1
딸기, 네가 궁금해                            1
Name: match_program, Length: 158, dtype: int64

#### Method 2'(5 words elimination) : TF-IDF Cosine Similarity

In [119]:
combined_tokens_1 = list(school_1['tokenized']) + list(program_1['tokenized'])
combined_tokens_1 = [' '.join(tokens) for tokens in combined_tokens_1]

vectorizer = TfidfVectorizer()
vectors_1 = vectorizer.fit_transform(combined_tokens_1)

tfidf_similarity_matrix_1 = cosine_similarity(vectors_1[:len(school)], vectors_1[len(school):])

In [120]:
tfidf_similarity_matrix_1.shape

(1366, 358)

In [121]:
max_indexes_1 = np.argmax(tfidf_similarity_matrix_1, axis=1)

In [122]:
result_tfidf_1 = school.copy()

In [123]:
result_tfidf_1['match_program'] = ""

for i in range(len(result_tfidf_1)):
  program_index_1 = max_indexes_1[i]
  result_tfidf_1['match_program'][i] = program['제목'][program_index_1]

<ipython-input-123-2c33a5208f8a>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_tfidf_1['match_program'][i] = program['제목'][program_index_1]


In [124]:
result_tfidf_1

,학교명,위치,학생수,예산(만원),매칭교과,강좌명,활동목표,위도,경도,제목_활동,tokenized,match_program
0,경북체육중학교,경산시,26.0,700.0,과학,내가 닮고 싶은 과학자 이야기,1. 과학자가 들려주는 과학 이야기를 읽고 내가 닮고 싶은 과학자에 대한 자료를 ...,35.812226,128.738053,내가 닮고 싶은 과학자 이야기 과학자가 들려주는 과학 이야기를 읽고 내가 닮고 싶...,"[과학자, 이야기, 과학자, 과학, 이야기, 과학자, 자료, 조사, 분석, 정리, ...","AI 창의과학(자율주행 자동차, 호버볼 만들기)"
1,경북체육중학교,경산시,26.0,700.0,국어,문학에 빠지다,"1. 문학에 대한 이해를 높이고, 책 읽기를 통하여 문학에 대한 이해를 기를 수 ...",35.812226,128.738053,문학에 빠지다 문학에 대한 이해를 높이고 책 읽기를 통하여 문학에 대한 이해를 기...,"[문학, 문학, 이해, 책, 문학, 이해, 독후, 다양, 활동, 표현, 문학, 감수]",적과한 사과를 이용하여 동물가족 표현하기
2,경북체육중학교,경산시,26.0,700.0,사회,영화 속 사회,"1. 일본군 위안부와 관련된 자료를 찾아 그 실상을 알고, 홍보 굿즈를 제작해 보...",35.812226,128.738053,영화 속 사회 일본군 위안부와 관련된 자료를 찾아 그 실상을 알고 홍보 굿즈를 제...,"[영화, 속, 사회, 일본군, 위안부, 관련, 자료, 실상, 홍보, 굿, 즈, 제작...",사회적기업가의 삶
3,경산여자중학교,경산시,281.0,NaN,국어,시 감상과 모방시 창작,1. 문학은 심미적 체험을 바탕으로 한 다양한 소통 활동임을 알고 문학 활동을 할...,35.817764,128.729945,시 감상과 모방시 창작 문학은 심미적 체험을 바탕으로 한 다양한 소통 활동임을 알...,"[시, 감상, 모방, 창작, 문학, 심미, 체험, 바탕, 다양, 소통, 활동, 문학...",창작이 즐거운 코딩
4,경산여자중학교,경산시,281.0,NaN,사회,내가 만들어가는 민주정치,1. 민주 정치를 실현하기 위한 노력을 국가와 시민의 차원에서 탐구하여 설명할 수 ...,35.817764,128.729945,내가 만들어가는 민주정치 민주 정치를 실현하기 위한 노력을 국가와 시민의 차원에서 ...,"[민주, 정치, 민주, 정치, 실현, 노력, 국가, 시민, 차원, 탐구, 설명, 다...","""DIY"" 자양 바로담아 고추장담그기"
...,...,...,...,...,...,...,...,...,...,...,...,...
1361,흥해중학교,포항시,151.0,1700.0,과학,나만의 과학용어 사전 만들기,1. 과학용어 사전 만들기 활동을 통해 과학의 기초용어를 이해할 수 있다.\n2. ...,36.109218,129.341171,나만의 과학용어 사전 만들기 과학용어 사전 만들기 활동을 통해 과학의 기초용어를 이...,"[과학, 용어, 사전, 과학, 용어, 사전, 활동, 과학, 기초, 용어, 이해, 자...",과학개념 완성 실험과학
1362,흥해중학교,포항시,151.0,1700.0,과학,창의력 넘치는 나만의 과학책,1. 과학책 만들기 활동을 통해 과학의 기초지식을 이해할 수 있다.\n2. 자신의 ...,36.109218,129.341171,창의력 넘치는 나만의 과학책 과학책 만들기 활동을 통해 과학의 기초지식을 이해할 수...,"[창의력, 나만, 과학, 책, 과학, 책, 활동, 과학, 기초, 지식, 이해, 자신...",과학개념 완성 실험과학
1363,흥해중학교,포항시,151.0,1700.0,정보,로봇 코딩,1. 로봇의 구조와 각 센서의 기능을 이해하고 코딩할 수 있다.\n2. 로봇의 여러...,36.109218,129.341171,로봇 코딩 로봇의 구조와 각 센서의 기능을 이해하고 코딩할 수 있다 로봇의 여러 가...,"[로봇, 코딩, 로봇, 구조, 센서, 기능, 이해, 코딩, 로봇, 동작, 기능, 제...",로봇공학자(중학생을 위한 창의융합 미래로봇 체험)
1364,흥해중학교,포항시,151.0,1700.0,영어,퍼니 멍키즈,1. 국가별 문화 차이나 다문화에 대한 정보를 수용하고 이해하는 태도를 기른다.\n...,36.109218,129.341171,퍼니 멍키즈 국가별 문화 차이나 다문화에 대한 정보를 수용하고 이해하는 태도를 기른...,"[키즈, 국가, 문화, 차, 문화, 정보, 수용, 이해, 태도, 영어, 정보, 문해...",3D펜 기초반
